# “Should This Loan be Approved or Denied?”

### Group Members: Dinesh Saraswat, Garima Vijay, Vinmathi Iyappan

# Executive Summary


This project focuses on building a predictive model to estimate the Probability of Default (PD) for small business loans using historical data from the U.S. Small Business Administration (SBA). The initiative supports regulatory compliance under the Basel Framework and aids banks in optimizing loan approvals for profitability and financial stability.
Small business loans inherently carry higher risk, making accurate PD predictions essential for:
1. Bank Decision-Making: Reducing financial losses and determining suitable loan terms.
2. Regulatory Compliance: Meeting Basel III Endgame's capital adequacy standards.
3. Profit Maximization: Optimizing returns while minimizing defaults.
Using SBA data spanning 1987–2014 (899,164 records, 27 features), we addressed challenges such as class imbalance, missing values, and data inconsistencies. A stratified sampling approach and robust data preprocessing pipelines ensured effective analysis and modeling.
The CatBoost model emerged as the best performer, delivering an average profit of $7,588.25 with a custom-tuned threshold (0.41). It demonstrated balanced performance across sensitivity, specificity, and profitability, providing an effective tool for loan decision-making.


# Procedures and Results

## Data Preparation

**Data cleaning:** 
1. **City and State Mapping**: Missing values in the City and State fields were filled using the first three digits of the ZIP code and external mapping data from the ‘L002 3-Digit ZIP Code Prefix Matrix.pdf’ to accurately understand the business geography.

2. **ZIP Code Standardization**: ZIP codes with a length of 4 were prefixed with a 0 to make them 5 digits, adhering to the standard ZIP code format.

3. **Approval Fiscal Year** (ApprovalFY)
     Alphabet Removal: Alphabetic characters in ApprovalFY were replaced with empty strings to clean the data.
     Type Conversion: The column was converted to integers for consistency.

4. **Standardized Numeric Columns**: Columns such as BalanceGross, DisbursementGross, ChgOffPrinGr, GrAppv, and SBA_Appv were converted to numeric by removing dollar signs and commas.

5. **LowDoc Cleanup**: Errors in the LowDoc column (expected to contain 'Y' or 'N') were corrected. Loans with approval amounts less than "150,000 dollars" were flagged as "Low Docs".

6. **Term Adjustment**: Terms less than 6 months were deemed erroneous so we updated to 6 months.

7. **MIS_Status Conversion**: Converted MIS_Status to binary (1 for defaulted, 0 for paid in full) for easier modeling and analysis.

8. **Records with missing values** in key fields like BankState, DisbursementDate, State, and City were dropped.

9. Missing values in the NewExists field were filled with 0(means undefined)

**New fields**

10. **SBA Portion Calculation**: A new field (SBA_portion) was created to represent the percentage of the approved gross amount covered by SBA.

11. **Recession Indicator**: A new field (recession) was created to flag loans with a maturity date between 2007-12-01 and 2009-06-30 as occurring during the recession.

12. **Real Estate Indicator**: Loans with a term greater than 240 months were flagged with a new RealEstate column, indicating they were backed by real estate.

13. **Franchise Flag**: A new field (If_Franchise) was created to indicate whether a loan was associated with a franchise (FranchiseCode not equal to 0 or 1).

14. **Revolving Line of Credit**: A new column (correct_RevLineCr) was created to flag loans as part of a revolving line of credit based on predefined codes.

15. **Industry Consolidation**: Industries based on the NAICS column were grouped into 7 major categories, consolidating the less frequent ones into an "Other" bucket to optimize category sizing and reduce complexity.

16. **DefaultRateBucket**: This column categorizes each state into buckets based on default rate levels.

17. **CompanySize**: Categorized businesses into size groups based on the number of employees.

    **Logic**:
    0: No employees (smallest businesses).
    1: Micro-businesses (1–5 employees).
    2: Small businesses (6–50 employees).
    3: Medium-sized businesses (51–250 employees).
    4: Large businesses (more than 250 employees).

18. **Job Created** - Group businesses by the number of jobs created

    **Logic**:
    0: No or minimal job creation (0–1 jobs).
    1: Small job creation (2–10 jobs).
    2: Moderate job creation (11–20 jobs).
    3: Significant job creation (more than 20 jobs).


**Predictors**: 

1. **categorical** = ['NewExist']
2. **label attributes** = [ 'IndustryFlag', 'CompanySize', 'DefaultRateBucket']
3. **log attributes** = ['updated_term', 'DisbursementGross']
4. **quantitative** = ['SBA_portion']
5. **pass(as they are already labeled)** = ['recession', 'RealEstate', 'LowDoc_correct', 'correct_RevLineCr', 'If_Franchise', 'JOB_CREATED']

    These variables are influential and consolidates most information from the given dataset

**Key Points About the Data Transformations Before the Model**

1. **Handling Categorical Variables:*
    *Label Encoding**:
     Columns 'IndustryFlag', 'CompanySize', 'DefaultRateBucket' are transformed into numerical format using LabelEncoder.
     Ensures compatibility with models that require numerical inputs while preserving categorical relationships.

2.	**One-Hot Encoding**:
    Columns 'NewExist', 'UrbanRural' are transformed into binary dummy variables using OneHotEncoder.
    Allows models to interpret categories as separate features without assuming ordinal relationships.

3. **Log Transformation for Highly Skewed Features**:
    Columns 'updated_term', 'DisbursementGross' , 'SBA_portion' are transformed using the Yeo-Johnson PowerTransformer.
    Handles positive and negative values, reducing skewness and stabilizing variance for highly skewed numeric data.

4. Standard Scaler is used to normalize predictors


## Data Sampling 

Stratified sampling was employed to reduce the dataset size while preserving the distribution of loan outcomes
and critical features that were highly imbalanced. This approach ensures that the sample maintains the same 
proportions of key variables as in the original dataset, which is particularly important when dealing with
imbalanced data, like loan defaults (MIS_Status) and other influential categorical variables.

Stratification was performed based on the following group of variables, which were highly imbalanced:
'NewExist', 'UrbanRural', 'LowDoc_correct', 'RealEstate', 'recession', 'If_Franchise', 
'correct_RevLineCr', 'CompanySize', 'JOB_CREATED', 'MIS_Status', 'DefaultRateBucket', 'IndustryFlag'.
By preserving the proportions of these variables in the sample, we ensure that the model is trained on a 
representative subset, which helps prevent biased learning and overfitting to the dominant class.

**Group by multiple attributes to ensure balanced sampling**
group_columns = ['NewExist', 'UrbanRural', 'LowDoc_correct', 'RealEstate', 'recession', 'If_Franchise',
                 'correct_RevLineCr', 'CompanySize', 'JOB_CREATED', 'MIS_Status', 'DefaultRateBucket', 'IndustryFlag']

Apply stratified sampling using Dask to reduce the dataset size (89348 (10% of the dataset)) while 
preserving key feature distributions.

## Defined Business Metric

We developed a custom business metric to evaluate financial impact:

1. Correct Approvals: 5% profit of loan amounts.
2. Incorrect Defaults: Losses at 5x loan amounts.
3. Denied Loans: 0

*Equation:*
1. denied_correctly =  0  # No profit or loss for denying defaults
2. denied_incorrectly = - 5 * 0.05 * disbursement_gross[false_negative].sum()  # Loss for incorrectly granting defaults
3. approved_incorectly = 0  # No profit or loss for denying Paid in Full
4. approved_correctly = (0.05 * disbursement_gross[true_negative]).sum()  # Profit for correctly granting Paid in Full

**total_profit = denied_correctly + denied_incorrectly + approved_incorectly + approved_correctly**

The **make_scorer** function in scikit-learn is used to create a custom scoring function that allowed us to define a 
performance metric tailored to the specific needs. 
business_scorer = make_scorer(business_metric).set_score_request(disbursement_gross=True)

To further optimize profitability, **TunedThresholdClassifierCV** from Scikit Learn is used to fine-tune the decision threshold based on a business-specific metric. This approach helped identify the optimal threshold, balancing sensitivity and profitability.

## Modeling and Results

Various machine learning models were evaluated using traditional performance metrics and the custom business metric. 
CatBoost outperformed all others, achieving:
    
**Default Threshold (0.5)**:
1. Business Metric: $7,554.98 average profit.
2. Accuracy: 91.83%
3. Sensitivity: 91.51%
4. Specificity: 91.91%
5. F1 Score: 79.51%

**Tuned Threshold (0.41)**:
1. Business Metric: $7,588.25 average profit.
2. Accuracy: 90.55%
3. Sensitivity: 93.22%
4. Specificity: 89.99%
5. F1 Score: 77.35%

**Best parameters**:

cat_model = CatBoostClassifier(
    iterations=500,               
    learning_rate=0.1,            
    depth=6,
    loss_function='Logloss',      
    eval_metric='AUC'
    random_seed=42,
    verbose=100,                  
    class_weights=[1, 5],         
)


# Output and Explanation

The optimized CatBoost model effectively predicts default probabilities for small business loans, balancing accuracy and profitability.

1. The tuned threshold of 0.41 delivers higher sensitivity (93.22%), improving default identification and reducing losses.

2. Profitability increased to $7,588.25 on average, demonstrating its practical utility for decision-making.

**Trade-Off Analysis**
The selection of a tuned threshold (0.41) was guided by the primary goal of maximizing profitability while maintaining strong predictive performance. This adjustment in the decision threshold demonstrates a deliberate trade-off between key metrics such as sensitivity, precision, and specificity to align the model's output with business objectives.

**Prioritizing Sensitivity**

1. Sensitivity, also known as the true positive rate, measures the model’s ability to correctly identify loan defaults.

2. A higher sensitivity at the tuned threshold (93.22% vs. 91.51% at 0.5) indicates that the model is better at flagging loans that are likely to default.

3. This improvement is crucial because missed defaults (false negatives) have the largest financial impact, representing loans that are incorrectly approved and result in substantial losses (5x the loan amount in this scenario).

By increasing sensitivity, the model effectively reduces the occurrence of false negatives, minimizing these high-cost mistakes. This prioritization directly translates into greater financial gains, as evidenced by the increase in the business metric (profit improved by $33.27 on average).

**Accepting Reductions in Precision and Specificity**
1. Precision measures how many predicted defaults were actual defaults. At the tuned threshold, precision dropped slightly from 70.29% to 66.09%, reflecting a higher number of false positives (loans flagged as defaults but not actually defaulting).

2. Specificity, which measures the ability to correctly identify non-defaults (loans likely to be fully paid), also saw a minor decrease from 91.91% to 89.99%.

These reductions indicate that the model occasionally misclassifies some loans as defaults that might have been successfully repaid. However, the financial impact of such false positives is negligible compared to the cost of missed defaults. Denying these loans has no associated profit or loss in the business metric, making this trade-off acceptable.

1. CatBoost clearly stands out with an exceptional ROC-AUC of 0.9713, indicating outstanding model performance.
2.  It outperforms other models not just in ROC-AUC, but also delivers the highest profitability of **7,588.25 dollars**, making it the best for business purposes.
3. It balances high accuracy, good sensitivity, and high specificity, which means it is both effective and reliable for predicting defaults without many false positives.


### Other Models 

1. **Logistic Regression Models**
Three logistic regression models were evaluated for predicting defaults:

*Baseline Logistic Regression (No Class Weights):*

**Performance Highlights**:
1. ROC-AUC: 0.8443.
2. Tuned Threshold (0.18): Improved sensitivity to 80.38% and profitability to $5,460.23.
3. While effective, it struggled to fully balance sensitivity and precision.
4. Logistic Regression with Class Weights ({0: 1, 1: 5}):

**Performance Highlights**:
1. ROC-AUC: 0.8489 (slight improvement over the baseline).
2. Tuned Threshold (0.54): Sensitivity of 78.31% and the highest business benefit of $5,619.83.
3. This model best addressed class imbalance, achieving the highest profitability and balanced performance.
4. Optimized Logistic Regression (ElasticNet Regularization):

**Performance Highlights**:
1. ROC-AUC: 0.8466.
2. Tuned Threshold (0.18): Sensitivity of 80.35% and profitability of $5,457.37.
3. Despite advanced tuning, performance gains over simpler models were marginal.

________________________________________

2. **KNN**

The kNN model underwent a series of improvements to enhance its prediction accuracy, particularly in identifying defaults.
1. Initially, with default settings and a threshold of 0.5, the model achieved an ROC-AUC of 0.8318, with high specificity (95.65%) but low sensitivity (47.40%), missing a significant number of defaults. The business benefit at this threshold was $4,697.87.

2. After tuning the threshold to 0.2, sensitivity improved dramatically to 82.88%, increasing the business benefit to $5,230.82. Further hyperparameter tuning identified the optimal k=9 with a Manhattan distance metric, which led to an accuracy of 87.68%, but sensitivity remained low at 46.02%.

3. However, the best model came from adjusting the threshold to 0.16, which yielded the highest sensitivity (77.58%) and an F1-Score of 60.32%. This model resulted in a business benefit of $5,686.10, showcasing the potential of kNN when optimized for specific business objectives, though it still underperformed compared to CatBoost in terms of overall performance.

________________________________________

3. **Random Forest**

1. The Random Forest model, using class weights to address the imbalanced nature of the dataset, performed well in terms of ROC-AUC and overall accuracy. Initially, with a cutoff threshold of 0.5, the model achieved an ROC-AUC score of 0.9554 and a business benefit of $6,070.57. The accuracy was 87.68%, but sensitivity was lower at 46.02%, indicating the model missed a significant number of defaults. The specificity was high (96.39%), demonstrating strong performance in predicting non-defaults.

2. When the decision threshold was tuned to 0.17, sensitivity increased substantially to 77.06%, reflecting the model's better ability to capture defaults. Precision also improved to 50.21%, and the F1-Score reached 0.6081, indicating a better balance between sensitivity and precision. The business benefit with the tuned threshold was $7,052.45, making it a more effective model for the goal of reducing defaults while still achieving acceptable specificity (84.02%).

3. This model demonstrated the importance of threshold tuning to balance sensitivity and specificity, improving performance for the classification task.
________________________________________
4. **XGBoost**

1. The XGBoost model, configured with a cost-sensitive setting (using scale_pos_weight=5 to account for the misclassification cost of class 1), demonstrated strong performance in terms of both ROC-AUC and business metrics. The model achieved an impressive ROC-AUC score of 0.9658, indicating excellent discrimination between classes. With the default cutoff threshold of 0.5, the model reached an accuracy of 91.84%, with high sensitivity (91.51%) and specificity (91.91%). Precision stood at 70.29%, and the F1-Score was 0.7951, reflecting a good balance between precision and recall.

2. When the decision threshold was tuned to 0.34, the model achieved even better sensitivity (94.87%), capturing more of the positive class, while precision decreased slightly to 61.74%. The F1-Score remained strong at 0.748, and specificity dropped slightly to 87.70%. The business benefit at the tuned threshold was slightly lower at 7,185.23 dollars, compared to 7,208.09 dollars with the default threshold.

3. Overall, the XGBoost model showed excellent performance, with sensitivity particularly benefiting from the threshold adjustment, making it highly effective for identifying defaults while maintaining a reasonable level of precision and specificity.
________________________________________
5. **Bagging Classifier**

1. The Bagging Classifier, utilizing a cost-sensitive Decision Tree as the base estimator, demonstrated strong performance in both ROC-AUC and business metrics. With a ROC-AUC score of 0.9591, the model effectively discriminated between the default and non-default classes. The default cutoff threshold of 0.5 resulted in an accuracy of 91.84%, with high sensitivity (91.51%) and specificity (91.91%). Precision was 70.29%, and the F1-Score was 0.7951.

2. When the decision threshold was adjusted to 0.19, the model showed a significant increase in sensitivity (97.37%), making it more sensitive to the positive class. However, this also resulted in a slight decrease in precision (51.27%) and specificity (80.64%). The F1-Score at the tuned threshold was 0.6717, indicating a more conservative approach with a better trade-off between recall and precision. The business benefit at the tuned threshold increased to $7,340.89 from $6,591.01 at the default threshold.

3. Overall, the Bagging Classifier performed well, with sensitivity greatly improving after the threshold tuning, making it a highly effective model for detecting defaults.
________________________________________
6. **Single Tree**

1. The cost-sensitive Decision Tree model, with a class weight of 5x for the positive class, performed well across several evaluation metrics, delivering strong results with a ROC-AUC score of 0.9506.

________________________________________
7. **Neural Network**

1. The neural network model performed reasonably well on the binary classification task, yielding an accuracy of 87.94% on the test set.

    o	Test Loss: 0.2934
    o	Test Accuracy: 87.94%

2. The neural network model shows strong performance with a high test accuracy of nearly 88%, but the recall and precision for detecting defaults (Class 1) could be improved. The business metric reflects this as well, with a slightly lower value compared to some other models.

________________________________________
8. **Linear Discriminant Analysis**

1. The LDA model has strong performance in detecting Class 0 (No Default) with high recall and precision but struggles with detecting defaults (Class 1), as indicated by the lower recall and precision for that class. The tuned threshold (0.18) improves the business metric, indicating better financial outcomes when adjusting for the imbalance between classes.



## Final model 


1. **Algorithm**: Catboost model ( train and test split - 70/30 ratio)
    
**Performance Metrics:**
    
Accuracy: 89.16%
Precision: 98% for non-default loans (class 0) and 71% for default loans (class 1).
Recall: 92% for both non-default and default loans.
F1-Score:
0.95 for non-default loans.
0.80 for default loans.

he model achieves a high recall for default loans (92%), critical for minimizing financial losses due to loan 
defaults.High precision for non-default loans (98%) ensures most approved loans generate profit.

**Business Metric Evaluation**

1. Threshold (Tuned): 0.44.
2. This threshold was determined to maximize business profit based on cumulative net profit.
3. Default Threshold Profit: 7,571.41 dollars.
4. Tuned Threshold Profit: 7,586.61 dollars.

Optimizing the threshold from the default value significantly improved business profit, demonstrating the 
importance of tailoring thresholds to business objectives.

**Profit Optimization**

1. Maximum Cumulative Profit: $134,036,359.15.
2. Optimal Cutoff Probability: 0.4056.
3. Optimal Number of Loans: 13,256.
    
These loans, selected using the tuned cutoff probability, balance the trade-off between granting loans 
to non-default borrowers and denying loans to potential defaulters.

# Answers Based on Model Results

**a. How far into the validation data should you go to get maximum net profit?**

To achieve the maximum cumulative net profit, the model suggests considering the first *13,256* loans from the validation dataset. These loans are ranked based on their predicted probabilities, prioritizing the least risky applications.

**b. If this model is used to score future loan applicants, what “probability of success” cut-off should be used in granting the loan and extending credit?**

The optimal "probability of success" cut-off for granting loans is *0.4056*. Loans with a predicted probability of success (non-default) greater than or equal to 40.56% should be approved to maximize net profit while minimizing risk.